#Multivariate LSTM model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import mysql.connector
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import joblib
import os


How to save and reload scaler to always use the same scale during training, validation, and prediction. This keeps the model’s outputs consistent and comparable across runs.

1️ Fit and Save the Global Scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

features_cols = ["market_value","total_injuries","sentiment_mean","avg_cards_per_match"]

# Fit once on your entire dataset
global_scaler = MinMaxScaler()
global_scaler.fit(df[features_cols].fillna(0).values)

# Save to disk
joblib.dump(global_scaler, "global_scaler.pkl")
print("✅ Scaler saved to global_scaler.pkl")

#This writes a binary file you can load later.

2️ Load the Scaler Later

In [ ]:
import joblib

# Load the same scaler back
global_scaler = joblib.load("global_scaler.pkl")
print("✅ Scaler loaded. Feature mins:", global_scaler.data_min_)

#This ensures the exact same min/max values are used for transforming new data.

3️ Use During Training

In [ ]:
# Inside the loop:
features = group[features_cols].fillna(0).values
scaled = global_scaler.transform(features)  # not fit_transform!

#Because we already fit it globally, we only call `.transform()` on each new subset.

4️ Use During Inference / Forecasting

In [ ]:
# to predict for one player:
features = player_df[features_cols].fillna(0).values
scaled = global_scaler.transform(features)

Xp, yp = make_multivariate_multistep(scaled, n_steps, n_future)
pred_scaled = model.predict(Xp)

# Inverse transform only the target column (market_value):
dummy = np.zeros((pred_scaled.shape[0], len(features_cols)))

for step in range(n_future):
    dummy[:,0] = pred_scaled[:,step]
    inv = global_scaler.inverse_transform(dummy)
    y_pred_original = inv[:,0]  # this is in actual market value

All training, validation, and prediction are on one consistent scale.
We can stop worrying about each run giving different min/max.
We can deploy your model and apply the same scaler on live data.
